In [1]:
# Imports and configuration
%load_ext autoreload
%autoreload 2

import pandas as pd
import src.utils.file_utils as fu
import src.utils.schema_utils as su
import src.helpers.clean_helpers as chelp

cfg_clean = fu.load_config("clean")
cfg_schema = fu.load_config("schema")

In [2]:
# Read in raw data file
raw_hmda_df = fu.load_parquet("hmda_raw")
print(raw_hmda_df.shape)

Loading dataset from /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/2024_combined_mlar_header.parquet
(12236879, 85)


In [3]:
# Drop columns that are not known prior to application except for target
features_to_drop = su.get_columns_by_attribute(cfg_schema, "role", "drop")
raw_hmda_df.drop(columns=features_to_drop, inplace=True)
print(raw_hmda_df.shape)

(12236879, 67)


In [4]:
# Strip leading and trailing whitespace
# All columns are currently string values, so we will perform this on all columns regardless of eventual datatype
chelp.strip_string_columns_inplace(raw_hmda_df)

In [5]:
# Check for null-like values. Using a 1% sample to speed up the check
NULL_LIKE = cfg_clean["clean"]["null_like"]
null_candidates = chelp.null_like_check(raw_hmda_df, NULL_LIKE)
null_candidates

Series([], Name: null_like_fraction, dtype: object)

In [6]:
# Check for columns that contain only null values
fully_null_cols = raw_hmda_df.columns[raw_hmda_df.isna().all()]
print("Columns with only null values: ", fully_null_cols.tolist())

Columns with only null values:  []


In [7]:
# Some columns have conflated both an "Exempt" flag and the feature value.  We need to identify these columns and separate the exempt flag into its own column

# Find the features using exempt
features = set(su.get_columns_by_attribute(cfg_schema, "role", "feature"))
exempt  = set(su.get_columns_by_attribute(cfg_schema, "exempt", True))
exempt_feature_cols = sorted(features & exempt) # We need the intersection so we exclude dropped columns
print("Features using exempt: ", exempt_feature_cols)

# Apply the transformation
created_flags = chelp.apply_exempt_split(raw_hmda_df, exempt_feature_cols)
print("Created: ", created_flags)

Features using exempt:  ['applicant_credit_scoring_model', 'balloon_payment', 'business_or_commercial_purpose', 'co_applicant_credit_scoring_model', 'combined_loan_to_value_ratio', 'county_code', 'debt_to_income_ratio', 'initially_payable_to_institution', 'interest_only_payment', 'intro_rate_period', 'loan_term', 'manufactured_home_secured_property_type', 'multifamily_affordable_units', 'negative_amortization', 'open_end_line_of_credit', 'other_non_amortizing_features', 'prepayment_penalty_term', 'property_value', 'reverse_mortgage', 'submission_of_application']
Created:  ['applicant_credit_scoring_model_exempt', 'balloon_payment_exempt', 'business_or_commercial_purpose_exempt', 'co_applicant_credit_scoring_model_exempt', 'combined_loan_to_value_ratio_exempt', 'county_code_exempt', 'debt_to_income_ratio_exempt', 'initially_payable_to_institution_exempt', 'interest_only_payment_exempt', 'intro_rate_period_exempt', 'loan_term_exempt', 'manufactured_home_secured_property_type_exempt', 'mu

In [8]:
# We have one exception case with income to clean up before conversion
raw_hmda_df["income"] = raw_hmda_df["income"].replace("999999999", pd.NA)

# Convert columns to correct data types
raw_hmda_df = chelp.convert_by_schema(raw_hmda_df, cfg_schema)
print(raw_hmda_df.dtypes)

preapproval: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
construction_method: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
lien_status: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
balloon_payment: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
interest_only_payment: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
negative_amortization: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
other_non_amortizing_features: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
reverse_mortgage: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
open_end_line_of_credit: failed to convert to Bool (AttributeError: module 'pandas' has no attribute 'to_bool')
business_or_commercial_purpose: f

In [9]:
# Identify which features are categorical
categorical = set(su.get_columns_by_attribute(cfg_schema, "type", "categorical"))
categorical_feature_cols = sorted(features & categorical) # We need the intersection so we exclude dropped columns
print("Categorical features: ", categorical_feature_cols)

Categorical features:  ['action_taken', 'applicant_age', 'applicant_age_above_62', 'applicant_credit_scoring_model', 'applicant_ethnicity_1', 'applicant_ethnicity_2', 'applicant_ethnicity_3', 'applicant_ethnicity_4', 'applicant_ethnicity_5', 'applicant_ethnicity_observed', 'applicant_race_1', 'applicant_race_2', 'applicant_race_3', 'applicant_race_4', 'applicant_race_5', 'applicant_race_observed', 'applicant_sex', 'applicant_sex_observed', 'balloon_payment', 'business_or_commercial_purpose', 'census_tract', 'co_applicant_age', 'co_applicant_age_above_62', 'co_applicant_credit_scoring_model', 'co_applicant_ethnicity_1', 'co_applicant_ethnicity_2', 'co_applicant_ethnicity_3', 'co_applicant_ethnicity_4', 'co_applicant_ethnicity_5', 'co_applicant_ethnicity_observed', 'co_applicant_race_1', 'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4', 'co_applicant_race_5', 'co_applicant_race_observed', 'co_applicant_sex', 'co_applicant_sex_observed', 'construction_method', 'county_c

In [10]:
# Convert columns to categorical
chelp.to_pandas_categoricals(raw_hmda_df, categorical_feature_cols)

In [11]:
# Checking for illogical or extreme numerical values
print("Negative/zero loan amounts: ", raw_hmda_df.query("loan_amount <= 0").shape[0])
print("Negative/zero property values: ", raw_hmda_df.query("property_value <= 0").shape[0])
print("Negative incomes: ", raw_hmda_df.query("income < 0").shape[0])

Negative/zero loan amounts:  0
Negative/zero property values:  0
Negative incomes:  7757


In [12]:
# Negative incomes appear to be typos, so correcting values to positive
mask = raw_hmda_df["income"] < 0
raw_hmda_df.loc[mask, "income"] = raw_hmda_df.loc[mask, "income"].abs()
print("Negative incomes: ", raw_hmda_df.query("income < 0").shape[0])

Negative incomes:  0


In [13]:
# Handling target variable action_taken
# There are three categories of codes: approved, denied, and excluded.  The excluded categories are cases where the application was never completed, a borrower withdrew an application before a lending decision, etc.

# Getting relevant configs
action_cfg = cfg_clean["clean"]["action_taken"]
approved = set(action_cfg["approved"])
denied = set(action_cfg["denied"])
valid = approved | denied

# Removes excluded observations and creates approved_flag target variable based on approved/denied
raw_hmda_df = chelp.apply_action_taken_flag(raw_hmda_df, cfg_clean)

# Sanity check result
counts = raw_hmda_df["denied_flag"].value_counts(dropna=False)
print("Approved/Denied breakdown:")
print(counts)

Approved/Denied breakdown:
denied_flag
False    6693164
True     2147948
Name: count, dtype: int64[pyarrow]


In [14]:
# Save updated data frame to disk
fu.save_parquet(raw_hmda_df, "hmda_2024_typed")

Saved to /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/hmda_2024_typed.parquet


PosixPath('/Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/hmda_2024_typed.parquet')

In [15]:
# Write CSV summary of ending schema
chelp.generate_schema_summary(raw_hmda_df, cfg_schema, path_key="schema_summary")

Wrote schema summary to /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/docs/schema_summary.csv  (88 columns)


,column_name,data_type,missing_pct,unique_count,sample_value,notes
0,activity_year,Int16,0.0,1,2024,
1,lei,string[pyarrow],0.0,4899,5493009V3WNJX9V2GZ85,
2,loan_type,category,0.0,4,1,
3,loan_purpose,category,0.0,6,1,
4,preapproval,category,0.0,2,2,
...,...,...,...,...,...,...
83,prepayment_penalty_term_exempt,bool[pyarrow],0.0,2,False,
84,property_value_exempt,bool[pyarrow],0.0,2,False,
85,reverse_mortgage_exempt,bool[pyarrow],0.0,2,False,
86,submission_of_application_exempt,bool[pyarrow],0.0,2,False,
